In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import sklearn

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 1. dropout 이 높으면 학습 노드의 감소로 학습에 악영향을 줄 수도 있다.
# 2. layer의 노드수 조정으로 베이스 라인을 넘음

In [4]:
# 랜덤 시드 설정
torch.manual_seed(1)
if device == "cuda" :
    torch.cuda.manual_seed_all(1)

In [7]:
# csv 데이터 로드
train = pd.read_csv("2021-ai-midterm-p1/train.csv")
test = pd.read_csv("2021-ai-midterm-p1/test.csv")
submission = pd.read_csv("2021-ai-midterm-p1/submit_sample.csv")

# 데이터 확인
print(train)

      index  0  1   2   3   4   5  6  7  8  ...  55  56  57  58  59  60  61  \
0         0  0  0   5  13   9   1  0  0  0  ...   0   0   0   6  13  10   0   
1         1  0  0   0  12  13   5  0  0  0  ...   0   0   0   0  11  16  10   
2         2  0  0   0   4  15  12  0  0  0  ...   0   0   0   0   3  11  16   
3         3  0  0   7  15  13   1  0  0  0  ...   0   0   0   7  13  13   9   
4         4  0  0   0   1  11   0  0  0  0  ...   0   0   0   0   2  16   4   
...     ... .. ..  ..  ..  ..  .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..   
1432   1792  0  0   4  10  13   6  0  0  0  ...   0   0   0   2  14  15   9   
1433   1793  0  0   6  16  13  11  1  0  0  ...   0   0   0   6  16  14   6   
1434   1794  0  0   1  11  15   1  0  0  0  ...   0   0   0   2   9  13   6   
1435   1795  0  0   2  10   7   0  0  0  0  ...   0   0   0   5  12  16  12   
1436   1796  0  0  10  14   8   1  0  0  0  ...   0   0   1   8  12  14  12   

      62  63  label  
0      0   0      0  
1      

In [8]:
# 사용하지 않은 컬럼을 제거하여 Train, Test 데이터 설정
x_train = np.array(train.drop(["index","label"],axis=1))
x_test = np.array(test.drop(["index"],axis=1))
y_train = np.array(train["label"])

In [9]:
# 데이터 전처리
# 이미 데이터가 정규화 되어 0~16사이의 값을 가지고 있음

In [11]:
# label 데이터 확인 
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [15]:
# 데이터 모양 확인
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(1437, 64)
(1437,)
(360, 64)


In [26]:
# 데이터 Tensor에 올리기
x_train = torch.Tensor(x_train).to(device)
x_test = torch.Tensor(x_test).to(device)
y_train = torch.LongTensor(y_train).to(device)

In [27]:
# 학습 파라미터 설정
lr = 0.001
epochs = 10000
drop_prob = 0.2

In [28]:
# 레이어 생성
layer1 = torch.nn.Linear(64,512,bias=True).to(device)
layer2 = torch.nn.Linear(512,512,bias=True).to(device)
layer3 = torch.nn.Linear(512,1024,bias=True).to(device)
layer4 = torch.nn.Linear(1024,10,bias=True).to(device)
relu = torch.nn.ReLU().to(device)
dropout = torch.nn.Dropout(drop_prob).to(device)

In [29]:
# 레이어 가중치 초기화
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)
torch.nn.init.xavier_normal_(layer4.weight)

Parameter containing:
tensor([[ 0.0596,  0.0306,  0.0293,  ..., -0.0427,  0.0134,  0.0364],
        [ 0.0468,  0.0705, -0.1309,  ...,  0.0140,  0.0094, -0.0156],
        [-0.0245,  0.0701,  0.0225,  ..., -0.0169, -0.0777, -0.0117],
        ...,
        [ 0.0673, -0.0098, -0.0804,  ...,  0.0517,  0.0285, -0.0075],
        [ 0.0172,  0.0112,  0.0133,  ...,  0.0413, -0.0032,  0.0136],
        [ 0.0130, -0.0259,  0.0191,  ...,  0.0266, -0.0119,  0.0188]],
       device='cuda:0', requires_grad=True)

In [30]:
# 모델 생성
model = torch.nn.Sequential(layer1, relu, dropout,
                           layer2, relu, dropout,
                           layer3, relu, dropout,
                           layer4).to(device)

In [31]:
# 손실 함수, 최적화 함수 정의
loss = torch.nn.CrossEntropyLoss().to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [35]:
# 학습
model.train()
for epoch in range(epochs+1):
    optim.zero_grad()
    h = model(x_train)
    cost = loss(h,y_train)
    cost.backward()
    optim.step()
    if epoch % (epochs/10) == 0:
        print(epoch, cost.item())

0 2.073924587620013e-09
1000 1.659821293742425e-07
2000 0.0
3000 7.63974639994558e-06
4000 0.0
5000 5.793015702693083e-07
6000 1.3521876063293803e-08
7000 2.4057509317998438e-09
8000 0.05896907299757004
9000 0.0
10000 0.0


In [43]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
    predict = torch.argmax(predict,dim=1)
    predict = predict.cpu().detach().numpy()
submission["label"] = predict
print(submission)

     index  label
0       23      3
1       29      9
2       30      0
3       32      5
4       44      7
..     ...    ...
355   1774      1
356   1784      5
357   1785      7
358   1786      9
359   1790      8

[360 rows x 2 columns]


In [45]:
submission.to_csv("submission.csv",index=False)